In [31]:
!pip install yapf

  Using cached yapf-0.32.0-py2.py3-none-any.whl (190 kB)


In [3]:
import pandas as pd
import pygwalker as pyg
import duckdb


# Add Titanic data set to a data frame with a Count value added to it
df = pd.read_csv(
    r"./data/Air_Traffic_Passenger_Statistics.csv"
    # "https://data.sfgov.org/resource/rkru-6vcg.csv"
)
# convert activity period to year-month format as column Year_Month
df = df.assign(
    Year_Month=pd.to_datetime(df["Activity Period"], format="%Y%m").apply(
        lambda x: x.strftime("%B-%Y")
    ),
    Year=pd.to_datetime(df["Activity Period"], format="%Y%m").dt.strftime("%Y"),
)

df["Operating Airline"] = df["Operating Airline"].str.replace(
    "United Airlines - Pre 07/01/2013", "United Airlines"
)


df.tail(4)

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count,Year_Month,Year
50726,202212,Virgin Atlantic,VS,Virgin Atlantic,VS,International,Europe,Deplaned,Other,International,A,5836,December-2022,2022
50727,202212,Virgin Atlantic,VS,Virgin Atlantic,VS,International,Europe,Enplaned,Other,International,A,6650,December-2022,2022
50728,202212,WestJet Airlines,WS,WestJet Airlines,WS,International,Canada,Deplaned,Other,International,A,1679,December-2022,2022
50729,202212,WestJet Airlines,WS,WestJet Airlines,WS,International,Canada,Enplaned,Other,International,A,1857,December-2022,2022


In [4]:
 df_summary_duckdb = duckdb.query('SELECT SUM("Passenger Count"), \
                                  "Year", "Operating Airline" \
                                  FROM df \
                                  group by "Year", "Operating Airline" \
                                  ').to_df()


In [6]:
select * from duckdb_extensions();

SyntaxError: invalid syntax (2574091799.py, line 1)

In [5]:
df_summary_duckdb.tail()

,"sum(""Passenger Count"")",Year,Operating Airline
863,258170.0,2020,EVA Airways
864,348518.0,2009,Hawaiian Airlines
865,886710.0,2018,British Airways
866,635392.0,2019,Emirates
867,221014.0,2010,Asiana Airlines


In [14]:
df_summary = (
    df[["Passenger Count", "Operating Airline", "Year"]]
    .groupby(["Year", "Operating Airline"])
    .agg(Passenger_Count=("Passenger Count", "sum"))
    .reset_index()
    .sort_values(["Year", "Passenger_Count"], ascending=[True, False])
    .groupby(["Year"], as_index=False, sort=False)
    .nth[:10]
)

In [15]:
df_summary.head()

,Year,Operating Airline,Passenger_Count
40,2005,United Airlines,15338298
9,2005,American Airlines,3438824
36,2005,SkyWest Airlines,2568754
18,2005,Delta Air Lines,2072072
39,2005,US Airways,1731352
